Let's Explore the data 

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
# load df_main from data folder
df_main = pd.read_csv('data/df_main_clean.csv')

In [33]:
df_main.dtypes

recipe_id                 object
recipe_name               object
category                  object
ratings                  float64
description               object
servings                   int64
total_time                object
ingredients               object
calories_kcal              int64
carbohydrateContent      float64
cholesterolContent        object
fiberContent              object
proteinContent           float64
saturatedFatContent       object
sodiumContent             object
sugarContent              object
fatContent               float64
unsaturatedFatContent     object
servingSize               object
dtype: object

In [64]:
df_main.sample(5)

,recipe_id,recipe_name,category,ratings,description,servings,total_time,ingredients,calories_kcal,carbohydrateContent,cholesterolContent,fiberContent,proteinContent,saturatedFatContent,sodiumContent,sugarContent,fatContent,unsaturatedFatContent,servingSize
11372,https://www.allrecipes.com/recipe/31778/ukrain...,Ukrainian Olha's Varenyky (Perogies),Dinner,4.4,Ukrainian Olha's Varenyky (perogies) These hal...,10,PT120M,"5 cups all-purpose flour,1 teaspoon salt,2 tab...",636,82.0,76 mg,7 g,16.0,17 g,919 mg,8 g,28.0,0 g,NaN
8572,https://www.allrecipes.com/recipe/235985/a-sco...,A Scotsman's Shepherd Pie,Dinner,4.8,This shepherd's pie is made with sharp Cheddar...,8,PT75M,"5 cups mashed, boiled potatoes,0.5 cup sour cr...",386,30.0,97 mg,3 g,17.0,11 g,561 mg,5 g,21.0,0 g,NaN
10492,https://www.allrecipes.com/recipe/17445/grille...,Grilled Asparagus,"Dinner,Side Dish",4.7,Grilled asparagus is easy to make in a matter ...,4,PT15M,"1 pound fresh asparagus spears, trimmed,1 tabl...",53,4.0,NaN,2 g,3.0,1 g,2 mg,2 g,4.0,0 g,NaN
2627,https://www.allrecipes.com/recipe/279878/rhuba...,Rhubarb Applesauce,Side Dish,NaN,"Apple cider, cinnamon, and fresh rhubarb provi...",16,PT35M,"8 cups peeled and thinly sliced apples,2 cups ...",87,22.0,NaN,2 g,0.0,NaN,4 mg,19 g,0.0,0 g,NaN
2304,https://www.allrecipes.com/recipe/264661/vanil...,Vanilla Cupcakes with Swiss Meringue Buttercream,Dessert,4.3,Try this vanilla cupcake recipe for two dozen ...,24,PT65M,"3 cups all-purpose flour,1.5 teaspoons baking ...",407,36.0,103 mg,0 g,4.0,18 g,144 mg,NaN,28.0,0 g,NaN


In [72]:
(df_main['ingredients'][1]).split(',')

['2 tablespoons vegetable oil',
 '1 pound beef sirloin',
 ' cut into 2-inch strips',
 '1.5 cups fresh broccoli florets',
 '1 red bell pepper',
 ' cut into matchsticks',
 '2 carrots',
 ' thinly sliced',
 '1 green onion',
 ' chopped',
 '1 teaspoon minced garlic',
 '2 tablespoons soy sauce',
 '2 tablespoons sesame seeds',
 ' toasted']

In [86]:
first = df_main['ingredients'][1].split(',')[3].split(' ') 

In [87]:
try:
    amount = float(first[0])
except:
    pass

unit = first[1]
ingredient = ' '.join(first[2:])

print(amount)

1.5


In [34]:
df_main.isnull().sum()

recipe_id                    0
recipe_name                  0
category                     0
ratings                    894
description                  0
servings                     0
total_time                   0
ingredients                  0
calories_kcal                0
carbohydrateContent          0
cholesterolContent        2177
fiberContent               419
proteinContent               0
saturatedFatContent        633
sodiumContent                7
sugarContent              1020
fatContent                   0
unsaturatedFatContent        0
servingSize              13335
dtype: int64

Splitting Data

In [35]:
df_nutrition = df_main[['recipe_name','ingredients','calories_kcal','carbohydrateContent','proteinContent', 'fatContent']]

In [36]:
# split the data into train and test
from sklearn.model_selection import train_test_split

X = df_nutrition[['ingredients']]
y = df_nutrition[['calories_kcal','carbohydrateContent','proteinContent', 'fatContent']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(10873, 1)
(2719, 1)
(10873, 4)
(2719, 4)


In [37]:
# create a pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('regressor', LinearRegression())
])

In [38]:
# fit the pipeline
pipeline.fit(X_train['ingredients'], y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('regressor', LinearRegression())])

In [39]:
# predict
y_pred = pipeline.predict(X_test['ingredients'])

In [65]:
y_pred.shape

(2719, 4)

In [66]:
# evaluate the model
from sklearn.metrics import mean_squared_error, r2_score

print('RMSE: ', mean_squared_error(y_test['calories_kcal'], y_pred[:,0], squared=False))
print('R2: ', r2_score(y_test['calories_kcal'], y_pred[:,0]))

RMSE:  202.8522146865819
R2:  0.05244808199677087


Percentage of the error

In [69]:
np.sqrt(np.mean((y_test['calories_kcal'].values - y_pred[:,0])**2))/np.mean(y_test['calories_kcal'].values)

0.606304464914605

In [43]:
# save the model
import joblib

joblib.dump(pipeline, 'models/nutrition_model.pkl')

['models/nutrition_model.pkl']

In [44]:
# load the model
nutrition_model = joblib.load('models/nutrition_model.pkl')

In [46]:
# predict
nutrition_model.predict(['1 egg,1 tablespoon water,1 tablespoon butter,1 tablespoon vegetable oil,1 onion, chopped,2 cups cooked white rice, cold,1 cup cooked, chopped chicken meat,2 tablespoons soy sauce,1 teaspoon ground black pepper'])

array([[432.31532765,  25.26769615,  29.91439933,  22.15837016]])

In [48]:
# perform a grid search
from sklearn.model_selection import GridSearchCV

parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'regressor__fit_intercept': [True, False]
}

grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train['ingredients'], y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('regressor', LinearRegression())]),
             n_jobs=-1,
             param_grid={'regressor__fit_intercept': [True, False],
                         'tfidf__max_df': (0.25, 0.5, 0.75),
                         'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             verbose=1)

In [49]:
# print the best parameters
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)
print(grid_search.best_estimator_.get_params())

# predict
y_pred = grid_search.predict(X_test['ingredients'])

# evaluate the model
print('RMSE: ', mean_squared_error(y_test, y_pred, squared=False))
print('R2: ', r2_score(y_test, y_pred))

{'regressor__fit_intercept': False, 'tfidf__max_df': 0.75, 'tfidf__ngram_range': (1, 3)}
0.26344304102778054
Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75, ngram_range=(1, 3))),
                ('regressor', LinearRegression(fit_intercept=False))])
{'memory': None, 'steps': [('tfidf', TfidfVectorizer(max_df=0.75, ngram_range=(1, 3))), ('regressor', LinearRegression(fit_intercept=False))], 'verbose': False, 'tfidf': TfidfVectorizer(max_df=0.75, ngram_range=(1, 3)), 'regressor': LinearRegression(fit_intercept=False), 'tfidf__analyzer': 'word', 'tfidf__binary': False, 'tfidf__decode_error': 'strict', 'tfidf__dtype': <class 'numpy.float64'>, 'tfidf__encoding': 'utf-8', 'tfidf__input': 'content', 'tfidf__lowercase': True, 'tfidf__max_df': 0.75, 'tfidf__max_features': None, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 3), 'tfidf__norm': 'l2', 'tfidf__preprocessor': None, 'tfidf__smooth_idf': True, 'tfidf__stop_words': None, 'tfidf__strip_accents': None, 'tfidf__sublinear_tf': False,

In [50]:
# save the model
joblib.dump(grid_search.best_estimator_, 'models/nutrition_model_gridsearch.pkl')

['models/nutrition_model_gridsearch.pkl']

In [51]:
# load the model
nutrition_model_gridsearch = joblib.load('models/nutrition_model_gridsearch.pkl')

In [52]:
# predict
nutrition_model_gridsearch.predict(['1 egg,1 tablespoon water,1 tablespoon butter,1 tablespoon vegetable oil,1 onion, chopped,2 cups cooked white rice, cold,1 cup cooked, chopped chicken meat,2 tablespoons soy sauce,1 teaspoon ground black pepper'])

array([[300.23394236,  31.82712335,  18.68416928,   9.85070849]])

In [53]:
# decission tree regressor
from sklearn.tree import DecisionTreeRegressor

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('regressor', DecisionTreeRegressor())
])

# fit the pipeline
pipeline.fit(X_train['ingredients'], y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('regressor', DecisionTreeRegressor())])

In [54]:
# predict
y_pred = pipeline.predict(X_test['ingredients'])

# evaluate the model
print('RMSE: ', mean_squared_error(y_test, y_pred, squared=False))
print('R2: ', r2_score(y_test, y_pred))

RMSE:  78.87145367315918
R2:  -0.4500631496550339


In [55]:
# save the model
joblib.dump(pipeline, 'models/nutrition_model_decisiontree.pkl')

['models/nutrition_model_decisiontree.pkl']

In [56]:
# load the model
nutrition_model_decisiontree = joblib.load('models/nutrition_model_decisiontree.pkl')

In [57]:
# predict
nutrition_model_decisiontree.predict(['1 egg,1 tablespoon water,1 tablespoon butter,1 tablespoon vegetable oil,1 onion, chopped,2 cups cooked white rice, cold,1 cup cooked, chopped chicken meat,2 tablespoons soy sauce,1 teaspoon ground black pepper'])

array([[440.,   8.,  10.,  42.]])

In [58]:
# perform a grid search
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'regressor__max_depth': [None, 5, 10, 20],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 5]
}

grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train['ingredients'], y_train)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('regressor', DecisionTreeRegressor())]),
             n_jobs=-1,
             param_grid={'regressor__max_depth': [None, 5, 10, 20],
                         'regressor__min_samples_leaf': [1, 2, 5],
                         'regressor__min_samples_split': [2, 5, 10],
                         'tfidf__max_df': (0.25, 0.5, 0.75),
                         'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             verbose=1)

In [59]:
# print the best parameters
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)
print(grid_search.best_estimator_.get_params())

{'regressor__max_depth': 10, 'regressor__min_samples_leaf': 5, 'regressor__min_samples_split': 2, 'tfidf__max_df': 0.5, 'tfidf__ngram_range': (1, 1)}
0.12638219254168642
Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.5)),
                ('regressor',
                 DecisionTreeRegressor(max_depth=10, min_samples_leaf=5))])
{'memory': None, 'steps': [('tfidf', TfidfVectorizer(max_df=0.5)), ('regressor', DecisionTreeRegressor(max_depth=10, min_samples_leaf=5))], 'verbose': False, 'tfidf': TfidfVectorizer(max_df=0.5), 'regressor': DecisionTreeRegressor(max_depth=10, min_samples_leaf=5), 'tfidf__analyzer': 'word', 'tfidf__binary': False, 'tfidf__decode_error': 'strict', 'tfidf__dtype': <class 'numpy.float64'>, 'tfidf__encoding': 'utf-8', 'tfidf__input': 'content', 'tfidf__lowercase': True, 'tfidf__max_df': 0.5, 'tfidf__max_features': None, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1), 'tfidf__norm': 'l2', 'tfidf__preprocessor': None, 'tfidf__smooth_idf': True, 'tfidf__stop_words

In [60]:
# predict
y_pred = grid_search.predict(X_test['ingredients'])

# evaluate the model
print('RMSE: ', mean_squared_error(y_test, y_pred, squared=False))
print('R2: ', r2_score(y_test, y_pred))

RMSE:  63.36685037323446
R2:  0.07955403584729373


In [61]:
# save the model
joblib.dump(grid_search.best_estimator_, 'models/nutrition_model_decisiontree_gridsearch.pkl')

['models/nutrition_model_decisiontree_gridsearch.pkl']

In [62]:
# load the model
nutrition_model_decisiontree_gridsearch = joblib.load('models/nutrition_model_decisiontree_gridsearch.pkl')

In [63]:
# predict
nutrition_model_decisiontree_gridsearch.predict(['1 egg,1 tablespoon water,1 tablespoon butter,1 tablespoon vegetable oil,1 onion, chopped,2 cups cooked white rice, cold,1 cup cooked, chopped chicken meat,2 tablespoons soy sauce,1 teaspoon ground black pepper'])

array([[268.02932961,  33.96578212,   5.35125698,  12.99581006]])